Just pretend you are the bioinformatician at the Covid-19 surveillance center of Wakanda (I chose a place that doesn't exist, because no one wants new waves or new variants)


We have sequenced six genomes and processed them as you learned yesterday.
Pangolin identifies our six genomes as part of lineage "U.3" (a sublineage of B.1.177, the UK variant / Kent variant / Alpha variant)

Are they a new lineage, instead? A new variant derived from the U.3? We will need to compare them with genomes that are available from public databases and see if they are eligible for a new Pango lineage proposal (see guidelines at https://www.pango.network/the-pango-nomenclature-system/statement-of-nomenclature-rules/)

We will download all available genomes and metadata from the GISAID database (for this practice, we will use a very narrow subset). We will use python and recall a couple of stand-alone programs to do this

Let's start!

First of all we will need to load python libraries

In [ ]:
import sys
from Bio import SeqIO
import pandas as pd
import numpy as np
import os
from Bio import AlignIO
import operator
from Bio import Phylo
import matplotlib as mpl
from Bio.Align import MultipleSeqAlignment
import collections

And open the metadata file as a dataframe using the library pandas (pd)

In [ ]:
df = pd.read_csv("/home/student/DATA/new_variants_data/Gisaid_metadata_subset.tsv", sep="\t")

print(df)

Now we need to select high quality genomes, i.e. :

a) complete

b) high coverage

c) low number of N positions. In our case, at most 0.1% of the genome (~30 nucleotides) is "N"

In [ ]:
df2=df[(df["Is complete?"] == True) & (df["Is high coverage?"] == True)]
df3=df2[(df2["N-Content"] <= 0.001) | (df2["N-Content"].isna()) ]
LL=df3["Virus name"].to_list()

print("We have selected %i genomes" %(len(LL)))

Now we build a fasta file with the selected genome. We will add the Pango lineage to the name (it will be useful to understand the phylogenetic tree)

We also add the reference genome and those that we sequenced in our lab

In [ ]:
outF=open("selected_for_phylogeny.fasta","w")

df3.set_index("Virus name",inplace=True)

for i in SeqIO.parse("/home/student/DATA/new_variants_data/Gisaid_sequences_subset.fasta","fasta"):
    if str(i.id).strip().split("|")[0] in LL:
        i.id=str(i.id).strip().split("|")[0]+"|"+str(df3.loc[str(i.id).strip().split("|")[0]]["Pango lineage"])
        SeqIO.write(i,outF,"fasta")

outF.close()


!cat Reference_SARS-CoV-2.fasta selected_for_phylogeny.fasta NEW.fasta > selected+reference.fasta

Let's take a look at our sequences now. We need to align them!

In [ ]:
#!seaview selected+reference.fasta
ll=[]
c=0
for i in SeqIO.parse("selected+reference.fasta", "fasta"):
    c+=1
    i.seq=i.seq[50:100]
    i.id=str(c)
    ll.append(i)

SeqIO.write(ll,sys.stdout, "phylip")

To align the sequences and visualize them again:

In [ ]:
!mafft --auto selected+reference.fasta > selected+reference.fasta.aln

#!seaview selected+reference.fasta.aln

ll=[]
c=0
for i in SeqIO.parse("selected+reference.fasta.aln", "fasta"):
    c+=1
    i.seq=i.seq[50:100]
    i.id=str(c)
    ll.append(i)
SeqIO.write(ll,sys.stdout, "phylip")

=======

And now build the phylogenetic tree, and visualize it

In [ ]:
!iqtree -mset HKY,TIM2,GTR -mfreq F -mrate G,R -alrt 1000 -bb 1000 -s selected+reference.fasta.aln -T 2 -redo > /dev/null

In [ ]:
mpl.rcParams['font.size'] = 8

mpl.rcParams['figure.figsize'] = (23, 23)
mpl.rcParams['figure.dpi'] = 300
mpl.rcParams['lines.linewidth'] = 1


tree = Phylo.read('selected+reference.fasta.aln.treefile', 'newick')
tree.root_with_outgroup("NC_045512.2")
Phylo.draw(tree)

#!seaview selected+reference.fasta.aln.treefile

...and extract the SNPs that originated the new clade

In [ ]:
####READ ALN

aln=AlignIO.read("selected+reference.fasta.aln","fasta")

##select the U.3 clade

aln2=MultipleSeqAlignment([])

for i in aln:
    if "U.3" in str(i.id):
        aln2.append(i)
        
##trim alignment 5'

for pos in range(aln2.get_alignment_length()):
    if not "-" in aln[:, pos]:
        p5p = pos
        break

##trim alignment 3'

for pos in reversed(range(aln2.get_alignment_length())):
    if not "-" in aln[:, pos]:
        p3p = pos
        break

aln2=aln2[:, p5p:p3p]

        
###EXTRACT SNPS and compare groups      

DICT={}
for i in range(0,len(aln2[0].seq)):
    POS=list(aln2[:, i])
    if len(set(POS))>1:
        our_clade=[]
        other_taxa=[]

        for y in range(0,len(aln2)):
            if "our_genome".upper() in str(aln2[y].id).upper():
                our_clade.append(aln2[y,i])
            else:
                other_taxa.append(aln2[y,i])
        OUR=max({x:our_clade.count(x) for x in list(set(our_clade))}.items(), key=operator.itemgetter(1))[0]
        OTHER=max({x:other_taxa.count(x) for x in list(set(other_taxa))}.items(), key=operator.itemgetter(1))[0]
        
        if OUR!=OTHER:
            #print(i,OUR,OTHER,our_clade,other_taxa)
            OU=collections.Counter(our_clade)
            OU=",".join([i.upper()+":"+str(OU[i]) for i in OU])
            OT=collections.Counter(other_taxa)
            OT=",".join([i.upper()+":"+str(OT[i]) for i in OT])
            DICT[i]={"PREVALENT_IN_CLADE":OUR.upper(),"PREVALENT_OUTSIDE":OTHER.upper(),"INSIDE_COUNT":OU, "OUTSIDE_COUNT":OT}
        
df=pd.DataFrame.from_dict(DICT, orient="index")
print(df)